# Model Training, a Decision Tree, Random Forest and XGBoost

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import tensorflow
pd.set_option('display.max_columns', None)

In [2]:
final_dataset = pd.read_csv("../data/finalDataset.csv")
final_dataset

,AGE_DIFF,ATP_POINTS_DIFF,ATP_RANK_DIFF,BEST_OF,DRAW_SIZE,ELO_DIFF,ELO_GRAD_LAST_100_DIFF,ELO_GRAD_LAST_10_DIFF,ELO_GRAD_LAST_200_DIFF,ELO_GRAD_LAST_25_DIFF,ELO_GRAD_LAST_3_DIFF,ELO_GRAD_LAST_50_DIFF,ELO_GRAD_LAST_5_DIFF,ELO_SURFACE_DIFF,H2H_DIFF,H2H_SURFACE_DIFF,HEIGHT_DIFF,N_GAMES_DIFF,P_1ST_IN_LAST_100_DIFF,P_1ST_IN_LAST_10_DIFF,P_1ST_IN_LAST_200_DIFF,P_1ST_IN_LAST_25_DIFF,P_1ST_IN_LAST_3_DIFF,P_1ST_IN_LAST_50_DIFF,P_1ST_IN_LAST_5_DIFF,P_1ST_WON_LAST_100_DIFF,P_1ST_WON_LAST_10_DIFF,P_1ST_WON_LAST_200_DIFF,P_1ST_WON_LAST_25_DIFF,P_1ST_WON_LAST_3_DIFF,P_1ST_WON_LAST_50_DIFF,P_1ST_WON_LAST_5_DIFF,P_2ND_WON_LAST_100_DIFF,P_2ND_WON_LAST_10_DIFF,P_2ND_WON_LAST_200_DIFF,P_2ND_WON_LAST_25_DIFF,P_2ND_WON_LAST_3_DIFF,P_2ND_WON_LAST_50_DIFF,P_2ND_WON_LAST_5_DIFF,P_ACE_LAST_100_DIFF,P_ACE_LAST_10_DIFF,P_ACE_LAST_200_DIFF,P_ACE_LAST_25_DIFF,P_ACE_LAST_3_DIFF,P_ACE_LAST_50_DIFF,P_ACE_LAST_5_DIFF,P_BP_SAVED_LAST_100_DIFF,P_BP_SAVED_LAST_10_DIFF,P_BP_SAVED_LAST_200_DIFF,P_BP_SAVED_LAST_25_DIFF,P_BP_SAVED_LAST_3_DIFF,P_BP_SAVED_LAST_50_DIFF,P_BP_SAVED_LAST_5_DIFF,P_DF_LAST_100_DIFF,P_DF_LAST_10_DIFF,P_DF_LAST_200_DIFF,P_DF_LAST_25_DIFF,P_DF_LAST_3_DIFF,P_DF_LAST_50_DIFF,P_DF_LAST_5_DIFF,WIN_LAST_100_DIFF,WIN_LAST_10_DIFF,WIN_LAST_200_DIFF,WIN_LAST_25_DIFF,WIN_LAST_3_DIFF,WIN_LAST_50_DIFF,WIN_LAST_5_DIFF,RESULT
0,-5.0,-1028.0,69.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,-5.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0
1,10.3,-257.0,126.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,8.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0
2,-3.7,-352.0,135.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0
3,-2.0,-19.0,7.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,-7.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0
4,-0.9,-734.0,162.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,-11.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

### Data Preprocessing, Scaling and Splitting, and Train-Test Split

In [3]:
# Convert data to numpy (exclude the first 5k matches, since ELO is not calculated well at this point
data = final_dataset.to_numpy(dtype=object)[5000:,:]
np.random.shuffle(data)

# Split the data using an 85% split between training and testing
split = 0.85
value = int(round(split*len(data)))

data_train = data[:value,:]
data_test = data[value:,:]

print("Training Data: "+str(data_train.shape))
print("Testing Data: "+str(data_test.shape))

Training Data: (76819, 68)
Testing Data: (13556, 68)


### Define mappers for the labels 

In [4]:
# Define several mappers
mapper = np.vectorize(lambda x: "Player 2 Wins" if x == 0 else "Player 1 Wins")
reverse_mapper = np.vectorize(lambda x: 0 if x == "Player 2 Wins" else 1)

# Training data
x_train = data_train[:,:-1]
y_pred_train = mapper(data_train[:,-1:]).squeeze()

# Testing data
x_test = data_test[:,:-1]
y_pred_test = mapper(data_test[:,-1:]).squeeze()

## Decision Tree

In [5]:
# Instantiate a Decision Tree
decision_sklearn = DecisionTreeClassifier(max_depth=4)
decision_sklearn = decision_sklearn.fit(x_train, y_pred_train)

# Make predictions and test accuracy
predictions_train = decision_sklearn.predict(x_train)
predictions_test = decision_sklearn.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

Train Accuracy: 0.6594201955245447
Test Accuracy: 0.6524786072587784


In [6]:
text_representation = tree.export_text(decision_sklearn, feature_names=final_dataset.columns[:-1])
print(text_representation)

|--- ELO_DIFF <= -30.86
|   |--- ELO_DIFF <= -194.98
|   |   |--- ELO_DIFF <= -282.68
|   |   |   |--- BEST_OF <= 4.00
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- BEST_OF >  4.00
|   |   |   |   |--- class: Player 2 Wins
|   |   |--- ELO_DIFF >  -282.68
|   |   |   |--- ELO_SURFACE_DIFF <= -101.88
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- ELO_SURFACE_DIFF >  -101.88
|   |   |   |   |--- class: Player 2 Wins
|   |--- ELO_DIFF >  -194.98
|   |   |--- ELO_SURFACE_DIFF <= -81.12
|   |   |   |--- AGE_DIFF <= -4.25
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- AGE_DIFF >  -4.25
|   |   |   |   |--- class: Player 2 Wins
|   |   |--- ELO_SURFACE_DIFF >  -81.12
|   |   |   |--- ATP_RANK_DIFF <= 92.50
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- ATP_RANK_DIFF >  92.50
|   |   |   |   |--- class: Player 2 Wins
|--- ELO_DIFF >  -30.86
|   |--- ELO_DIFF <= 108.07
|   |   |--- ATP_RANK_DIFF <= -3.50
|   |   |   |--- ELO_SURFACE_DIFF <= -36.8

## Random Forest

In [8]:
# Instantiate a Random Forest
print("Training Random Forest Classifier...")
forest_sklearn = RandomForestClassifier(n_estimators=500, max_depth=10, max_features="sqrt", bootstrap=True)
forest_sklearn = forest_sklearn.fit(x_train, y_pred_train)
print("Random Forest Classifier trained successfully.")

# Make predictions and test accuracy
predictions_train = forest_sklearn.predict(x_train)
predictions_test = forest_sklearn.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

Training Random Forest Classifier...
Random Forest Classifier trained successfully.
Train Accuracy: 0.7120634218097085
Test Accuracy: 0.66737975804072


In [23]:
# Instantiate a Random Forest
forest_sklearn2 = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_split=400, min_samples_leaf=250, max_features="sqrt", bootstrap=True)
forest_sklearn2 = forest_sklearn2.fit(x_train, y_pred_train)

# Make predictions and test accuracy
predictions_train = forest_sklearn2.predict(x_train)
predictions_test = forest_sklearn2.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

KeyboardInterrupt: 

In [7]:
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [10, 20],
    'min_samples_leaf': [5, 10],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_grid,
    cv=5, 
    n_jobs=-1,
    verbose=4
)
grid_search.fit(x_train, y_pred_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [5, 10],
                         'min_samples_split': [10, 20],
                         'n_estimators': [100, 300]},
             verbose=4)

In [9]:
# Best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Instantiate a Random Forest
best_forest_model = RandomForestClassifier(max_depth=15, max_features='log2', min_samples_leaf=10, min_samples_split=10, n_estimators=300)
best_forest_model = best_forest_model.fit(x_train, y_pred_train)

# Make predictions and test accuracy
predictions_train = best_forest_model.predict(x_train)
predictions_test = best_forest_model.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

Train Accuracy: 0.7874744529348208
Test Accuracy: 0.6659043965771614


## XGBoost

In [11]:
# Instantiate a XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=200, max_depth=10, learning_rate=0.1, subsample=0.8, colsample_bytree=0.7)

# Train the model
xgb_model.fit(x_train, reverse_mapper(y_pred_train))

# Make predictions
predictions_train = xgb_model.predict(x_train)
predictions_test = xgb_model.predict(x_test)

# Calculate accuracy
print("Train Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_train), predictions_train)))
print("Test Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_test), predictions_test)))

Train Accuracy: 0.9734050169879848
Test Accuracy: 0.6603717910888167


In [12]:
# Sorting by importance in descending order
sorted_mapped_results = sorted(
    list(zip(final_dataset.columns[:-1], list(xgb_model.feature_importances_))),
    key=lambda x: x[1], 
    reverse=True
)

# Extracting sorted labels and their importance
sorted_labels = [label for label, importance in sorted_mapped_results]
sorted_importance = [importance for _, importance in sorted_mapped_results]

# Displaying results
for label, importance in sorted_mapped_results:
    print(f"{label}: {importance:.4f}")


ELO_DIFF: 0.0733
ELO_SURFACE_DIFF: 0.0522
ATP_POINTS_DIFF: 0.0219
BEST_OF: 0.0209
ATP_RANK_DIFF: 0.0172
AGE_DIFF: 0.0158
ELO_GRAD_LAST_200_DIFF: 0.0148
P_2ND_WON_LAST_50_DIFF: 0.0141
WIN_LAST_200_DIFF: 0.0140
H2H_SURFACE_DIFF: 0.0138
P_1ST_WON_LAST_10_DIFF: 0.0137
P_1ST_IN_LAST_25_DIFF: 0.0137
P_1ST_WON_LAST_25_DIFF: 0.0137
H2H_DIFF: 0.0137
P_1ST_WON_LAST_3_DIFF: 0.0136
P_1ST_IN_LAST_50_DIFF: 0.0135
ELO_GRAD_LAST_50_DIFF: 0.0135
P_2ND_WON_LAST_25_DIFF: 0.0135
ELO_GRAD_LAST_100_DIFF: 0.0134
P_1ST_IN_LAST_200_DIFF: 0.0134
WIN_LAST_3_DIFF: 0.0134
P_2ND_WON_LAST_10_DIFF: 0.0134
WIN_LAST_25_DIFF: 0.0133
P_2ND_WON_LAST_100_DIFF: 0.0133
P_1ST_WON_LAST_50_DIFF: 0.0133
P_ACE_LAST_3_DIFF: 0.0132
N_GAMES_DIFF: 0.0132
P_1ST_WON_LAST_200_DIFF: 0.0132
P_DF_LAST_100_DIFF: 0.0132
P_BP_SAVED_LAST_200_DIFF: 0.0131
P_DF_LAST_25_DIFF: 0.0131
ELO_GRAD_LAST_3_DIFF: 0.0131
P_1ST_WON_LAST_5_DIFF: 0.0131
P_BP_SAVED_LAST_100_DIFF: 0.0131
P_ACE_LAST_200_DIFF: 0.0131
P_DF_LAST_50_DIFF: 0.0131
P_BP_SAVED_LAST_50_D

In [14]:
# Instantiate an XGBoost Classifier
xgb_model = XGBClassifier(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.05, 
    subsample=0.7, 
    colsample_bytree=0.6,
    reg_alpha=0.1,
    reg_lambda=1.0
)

# Train the model
xgb_model.fit(x_train, reverse_mapper(y_pred_train))

# Make predictions
predictions_train = xgb_model.predict(x_train)
predictions_test = xgb_model.predict(x_test)

# Calculate accuracy
print("Train Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_train), predictions_train)))
print("Test Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_test), predictions_test)))

Train Accuracy: 0.6801832879886487
Test Accuracy: 0.6697403363824137


In [22]:
# Sorting by importance in descending order
sorted_mapped_results = sorted(
    list(zip(final_dataset.columns[:-1], list(xgb_model.feature_importances_))),
    key=lambda x: x[1], 
    reverse=True
)

# Extracting sorted labels and their importance
sorted_labels = [label for label, importance in sorted_mapped_results]
sorted_importance = [importance for _, importance in sorted_mapped_results]

# Displaying results
for label, importance in sorted_mapped_results:
    print(f"{label}: {importance:.4f}")

ELO_DIFF: 0.2108
ELO_SURFACE_DIFF: 0.1261
P_2ND_WON_LAST_100_DIFF: 0.0470
ATP_POINTS_DIFF: 0.0456
ATP_RANK_DIFF: 0.0268
WIN_LAST_50_DIFF: 0.0207
WIN_LAST_25_DIFF: 0.0188
AGE_DIFF: 0.0166
P_2ND_WON_LAST_10_DIFF: 0.0143
BEST_OF: 0.0137
P_2ND_WON_LAST_200_DIFF: 0.0136
P_2ND_WON_LAST_50_DIFF: 0.0131
N_GAMES_DIFF: 0.0128
P_1ST_WON_LAST_10_DIFF: 0.0119
P_1ST_WON_LAST_50_DIFF: 0.0118
WIN_LAST_100_DIFF: 0.0114
P_1ST_WON_LAST_3_DIFF: 0.0111
P_1ST_WON_LAST_5_DIFF: 0.0110
P_2ND_WON_LAST_25_DIFF: 0.0106
WIN_LAST_5_DIFF: 0.0100
P_2ND_WON_LAST_5_DIFF: 0.0095
DRAW_SIZE: 0.0093
H2H_DIFF: 0.0091
P_1ST_IN_LAST_3_DIFF: 0.0088
P_ACE_LAST_3_DIFF: 0.0087
P_2ND_WON_LAST_3_DIFF: 0.0087
P_DF_LAST_3_DIFF: 0.0086
P_DF_LAST_5_DIFF: 0.0086
P_1ST_IN_LAST_5_DIFF: 0.0083
P_1ST_WON_LAST_100_DIFF: 0.0083
WIN_LAST_10_DIFF: 0.0082
P_ACE_LAST_10_DIFF: 0.0081
P_BP_SAVED_LAST_100_DIFF: 0.0078
P_1ST_WON_LAST_25_DIFF: 0.0076
P_1ST_IN_LAST_10_DIFF: 0.0075
ELO_GRAD_LAST_200_DIFF: 0.0074
ELO_GRAD_LAST_100_DIFF: 0.0074
P_BP_SAVED

In [14]:
xgb_model.save_model("../models/xgb_model.json")

In [25]:
# Define parameter grid with all specified parameters
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05],
    'subsample': [0.7],
    'colsample_bytree': [0.6],
    'reg_alpha': [0.1, 0.5],
    'reg_lambda': [0.5, 1.0]
}

# Instantiate an XGBoost Classifier
xgb_model = XGBClassifier()

# Perform GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model, 
    param_grid=param_grid, 
    scoring='accuracy', 
    cv=5, 
    verbose=3, 
    n_jobs=-1
)
grid_search.fit(x_train, reverse_mapper(y_pred_train))

Fitting 5 folds for each of 72 candidates, totalling 360 fits


PicklingError: Could not pickle the task to send it to the workers.

In [13]:
# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Train the best model
best_xgb_model = XGBClassifier(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.05, 
    subsample=0.7, 
    colsample_bytree=0.6,
    reg_alpha=0.1,
    reg_lambda=0.5
)

best_xgb_model.fit(x_train, reverse_mapper(y_pred_train))

# Make predictions
predictions_train = best_xgb_model.predict(x_train)
predictions_test = best_xgb_model.predict(x_test)

# Calculate accuracy
print("Train Accuracy:", accuracy_score(reverse_mapper(y_pred_train), predictions_train))
print("Test Accuracy:", accuracy_score(reverse_mapper(y_pred_test), predictions_test))

Train Accuracy: 0.6808602038558169
Test Accuracy: 0.6684125110652109


In [15]:
# Sorting by importance in descending order
sorted_mapped_results = sorted(
    list(zip(final_dataset.columns[:-1], list(best_xgb_model.feature_importances_))),
    key=lambda x: x[1], 
    reverse=True
)

# Extracting sorted labels and their importance
sorted_labels = [label for label, importance in sorted_mapped_results]
sorted_importance = [importance for _, importance in sorted_mapped_results]

# Displaying results
for label, importance in sorted_mapped_results:
    print(f"{label}: {importance:.4f}")

ELO_DIFF: 0.2044
ELO_SURFACE_DIFF: 0.1237
ATP_POINTS_DIFF: 0.0578
P_2ND_WON_LAST_100_DIFF: 0.0479
WIN_LAST_50_DIFF: 0.0235
ATP_RANK_DIFF: 0.0218
P_2ND_WON_LAST_25_DIFF: 0.0185
AGE_DIFF: 0.0167
P_2ND_WON_LAST_50_DIFF: 0.0155
P_2ND_WON_LAST_200_DIFF: 0.0137
BEST_OF: 0.0133
P_1ST_WON_LAST_5_DIFF: 0.0132
N_GAMES_DIFF: 0.0118
P_2ND_WON_LAST_10_DIFF: 0.0117
WIN_LAST_25_DIFF: 0.0116
P_1ST_WON_LAST_100_DIFF: 0.0115
P_1ST_WON_LAST_10_DIFF: 0.0114
P_1ST_WON_LAST_3_DIFF: 0.0109
P_1ST_WON_LAST_50_DIFF: 0.0106
P_1ST_WON_LAST_25_DIFF: 0.0102
WIN_LAST_5_DIFF: 0.0100
WIN_LAST_100_DIFF: 0.0099
P_2ND_WON_LAST_5_DIFF: 0.0091
P_ACE_LAST_3_DIFF: 0.0089
P_1ST_IN_LAST_200_DIFF: 0.0085
P_2ND_WON_LAST_3_DIFF: 0.0084
H2H_DIFF: 0.0083
P_1ST_IN_LAST_3_DIFF: 0.0082
ELO_GRAD_LAST_100_DIFF: 0.0081
P_1ST_IN_LAST_10_DIFF: 0.0080
H2H_SURFACE_DIFF: 0.0080
P_DF_LAST_3_DIFF: 0.0079
ELO_GRAD_LAST_10_DIFF: 0.0077
ELO_GRAD_LAST_25_DIFF: 0.0076
P_ACE_LAST_50_DIFF: 0.0076
WIN_LAST_10_DIFF: 0.0076
ELO_GRAD_LAST_3_DIFF: 0.0075
P

In [17]:
best_xgb_model = grid_search.best_estimator_
best_xgb_model.save_model("../models/final_xgb_model.json")